In [29]:
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.chat_models import init_chat_model
import os
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest


load_dotenv()

True

In [30]:
@dataclass
class User:
    username: str

In [31]:
llm = init_chat_model("openai:gpt-4o")
token = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")

In [32]:
client = MultiServerMCPClient(
        {
            "github": {
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-github"],
                "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": token},
                "transport": "stdio",
            }
        }
    )
print("MCP Client created successfully")
tools = await client.get_tools()
# print(tools)

MCP Client created successfully


In [49]:
@dynamic_prompt
def user_context_prompt(request: ModelRequest) -> str:
    """Dynamic system prompt using runtime context."""
    username = "unknown"
    if hasattr(request.runtime.context, 'username'):
        username = request.runtime.context.username

    system_prompt = f"""You have access to GitHub tools. When the user asks about GitHub data 
(commits, repos, issues, etc.), ALWAYS use the available GitHub tools to fetch real data.
Never say you can't access this information - use the tools provided.

IMPORTANT: The authenticated GitHub user is `{username}`. When the user says "my commits", 
"your commits", or asks about "you" in a GitHub context, they mean the GitHub user `{username}`, 
NOT you (the AI assistant). Always query GitHub for user `{username}` when asked about 
personal GitHub activity."""

    return system_prompt

In [50]:
agent = create_agent(
    llm,
    tools,
    context_schema=User,
    middleware=[user_context_prompt]
)

In [55]:
res = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "use github tool and tell me how many github commits you have done in 2026?"}]},
    context=User(username="aayushmaanhooda")
)

try:
    res = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "List all repositories for user aayushmaanhooda on GitHub"}]},
        context=User(username="aayushmaanhooda")
    )
except Exception as e:
    print(f"Error: {e}")
    
# Check intermediate messages before the error
for msg in res.get("messages", []):
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print("Tool call attempted:", msg.tool_calls)


Tool call attempted: [{'name': 'search_repositories', 'args': {'query': 'user:aayushmaanhooda'}, 'id': 'call_O1vX8TkxcvWj8JIot21iZp6L', 'type': 'tool_call'}]
